In [6]:
# Machine Learning Exploration Project
### Author: Brandon Jean-Baptiste


In [7]:
# Importing necessary libraries for data manipulation, visualization, and modeling
import numpy as np                  # for numerical operations
import pandas as pd                 # for data handling
import matplotlib.pyplot as plt     # for plotting charts
import seaborn as sns               # for enhanced plotting
from scipy import stats             # for statistical operations
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
